In [8]:
import pandas as pd
# from datetime import datetime, date, time
import config, requests, json, random, time
import tweepy

auth = tweepy.OAuthHandler(consumer_key=config.API_KEY,
                           consumer_secret=config.API_KEY_SECRET)
auth.set_access_token(config.ACCESS_TOKEN, config.ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

In [9]:
user_input = input("remote or local?")

if user_input.lower() == "remote":
    data = pd.read_json('https://data.cityofchicago.org/resource/4ijn-s7e5.json')
elif user_input.lower() == "local":
    data = pd.read_csv('Food_Inspections.csv')
else:
    raise ValueError("Invalid option. Valid answers are 'remote' or 'local'.")


In [10]:
#fix column headers
data.columns = data.columns.str.strip()
standardize_columns = {'Inspection ID':'inspection_id', 'DBA Name':'dba_name', 'AKA Name':'aka_name', 'License #':'license_', 'Facility Type':'facility_type',
       'Risk':'risk', 'Address':'address', 'City':'city', 'State':'state', 'Zip':'zip', 'Inspection Date':'inspection_date',
       'Inspection Type':'inspection_type', 'Results':'results', 'Violations':'violations', 'Latitude':'latitude', 'Longitude':'longitude',
       'Location':'location'
}
data.rename(columns=standardize_columns, inplace=True)

In [11]:
if user_input.lower() == "remote":
    data['inspection_date'] = pd.to_datetime(data['inspection_date'], format='%Y-%m-%d')
else:
    data['inspection_date'] = pd.to_datetime(data['inspection_date'], format='%Y/%m/%d/')

data['clean_year'] = data['inspection_date'].dt.strftime('%Y')
data['clean_date'] = data['inspection_date'].dt.strftime('%m/%d/%Y')

most_recent_date = data['inspection_date'].max()
data['results'].unique()
data['results'].value_counts()



Pass                  491
Fail                  217
Pass w/ Conditions    126
No Entry               71
Out of Business        65
Not Ready              30
Name: results, dtype: int64

In [12]:
recent_results = data[data['inspection_date'] == most_recent_date]
recent_results['results'].value_counts()

Pass         4
Fail         2
Not Ready    1
Name: results, dtype: int64

In [13]:
#build the statuses

status_list = []

for i, current_restaurant in recent_results.iterrows():
    
    current_result = current_restaurant['results']
    current_name = current_restaurant['dba_name']
    current_address = current_restaurant['address']
    current_date = current_restaurant['clean_date']


    current_status = ''

    match current_result:
        case "Pass":
            current_status = (f'{current_name} at {current_address} passed the health inspection on {current_date}')
        case "Fail":
            current_status = (f'{current_name} at {current_address} failed the health inspection on {current_date}')
        case "Pass w/ Conditions":
            current_status = (f'{current_name} at {current_address} conditionally passed the health inspection {current_date}. Buyer beware.')
        case "Out of Business":
            current_status = (f'{current_name} at {current_address} appears to be Out of Business')
        case "No Entry":
            current_status = (f'No Entry could occur at {current_name} at {current_address} ...')
        case _:
            current_status = (f'The rating at {current_name} at {current_address} on {current_date} is unavailable.')

    status_list.append(current_status)

In [14]:
for x in status_list:
    
    time.sleep(30)

    api.update_status(x)
